# imports

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm, nbinom, gamma
from datetime import timedelta
import psycopg2

In [2]:
conn_params = {
    "host" : "srvdados",
    "database" : "postgres",
    "user":"compras",
    "password": "pecist@compr@s2024"
}

In [3]:
ve = """
    SELECT 
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date AS dt_emissao,
    SUM(pp.qtde_ven) AS vendas
FROM "D-1".prod_ped pp
JOIN "D-1".cliente c 
    ON pp.codcli = c.codcli
WHERE 
    pp.tipped = 'V'
    AND pp.dt_emissao >= DATE '2019-01-01'
    AND pp.cd_loja NOT IN ('01','08','09')
    and c.codcli not in ('13996','16100','18400','20000','23000','02608','24000','00270','20690','20691','20692','20693','23011','99999','88888','21097')
    and pp.codvde not in ('0100','0001','0006','2319')
    AND c.codarea <> '112'
    AND c.codcid <> '0501'
GROUP BY
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date;
"""

In [4]:
dv = """
   SELECT 
    pe.cd_produto AS cod_pro,
    pe.cd_loja,
    e.dt_emissao::date AS dt_emissao,
    SUM(pe.qt_devolve) AS devolucoes
FROM "D-1".prod_ent pe
JOIN "D-1".entrada e
    ON e.cd_loja = pe.cd_loja
    AND e.sg_serie = pe.sg_serie
    AND e.nu_nota = pe.nu_nota
JOIN "D-1".cliente cli
    ON cli.codcli = pe.cd_cliente
WHERE
    e.dt_emissao >= DATE '2019-01-01'
    AND e.in_cancela = 'N'
    AND e.in_clifor = 'C'
    AND UPPER(e.nfeenvstat) NOT LIKE '%DENEG%'
    AND pe.cd_cfop NOT IN ('1949', '2949', '1603')
    AND cli.codcli NOT IN ('99999','88888','21097')
    AND cli.codcid <> '0501'
    AND cli.codarea <> '112'
    AND pe.cd_loja NOT IN ('01','08','09')
GROUP BY
    pe.cd_produto,
    pe.cd_loja,
    e.dt_emissao::date;
"""

In [5]:
grid = """
select codpro, qt_grid, cd_loja from "D-1".prd_loja 
where cd_loja not in ('08','09','01')
"""

In [6]:
with psycopg2.connect(**conn_params) as conn:
    df_grid = pd.read_sql(grid, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_12232\24163732.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_grid = pd.read_sql(grid, conn)


In [7]:
with psycopg2.connect(**conn_params) as conn:
    ven = pd.read_sql(ve, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_12232\2245771844.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ven = pd.read_sql(ve, conn)


In [8]:
with psycopg2.connect(**conn_params) as conn:
    dev = pd.read_sql(dv, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_12232\2747580985.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dev = pd.read_sql(dv, conn)


In [9]:
LEAD_TIME = 30
DATA_INICIO_SIM = "2025-01-01"
ALVO_LOSS = 0.05


def norm_codpro(s: pd.Series) -> pd.Series:
    return (s.astype("string")
             .str.replace(r"\.0$", "", regex=True)
             .str.strip()
             .str.zfill(6))

def norm_loja(s: pd.Series) -> pd.Series:
    return s.astype("string").str.strip()

In [10]:
ven['dt_emissao'] = pd.to_datetime(ven['dt_emissao'])
dev['dt_emissao'] = pd.to_datetime(dev['dt_emissao'])

In [11]:
ven["cod_pro"] = norm_codpro(ven["cod_pro"])
dev["cod_pro"] = norm_codpro(dev["cod_pro"])
ven["cd_loja"]  = norm_loja(ven["cd_loja"])
dev["cd_loja"]  = norm_loja(dev["cd_loja"])

df = ven.merge(dev, on=["cod_pro", "cd_loja", "dt_emissao"], how="outer")

df["vendas"] = pd.to_numeric(df["vendas"], errors="coerce").fillna(0).astype("float32")
df["devolucoes"] = pd.to_numeric(df["devolucoes"], errors="coerce").fillna(0).astype("float32")
df["demanda_dia"] = (df["vendas"] - df["devolucoes"]).clip(lower=0).astype("float32")

In [12]:
ini_2025 = pd.to_datetime("2025-01-01")
fim_2025 = pd.to_datetime("2025-12-31")

alto_giro_prod_loja = (
    df.loc[(df["dt_emissao"] >= ini_2025) & (df["dt_emissao"] <= fim_2025),
           ["cod_pro", "cd_loja", "demanda_dia"]]
      .groupby(["cod_pro", "cd_loja"], as_index=False, observed=True)
      .agg(demanda_liquida_2025=("demanda_dia", "sum"))
      .query("demanda_liquida_2025 > 108")
      [["cod_pro", "cd_loja"]]
)

print("Pares produto+loja alto giro (demanda líquida 2025 > 108):", len(alto_giro_prod_loja))

# aplica o filtro no df inteiro (mantém histórico 2019+ mas só desses pares)
df = df.merge(alto_giro_prod_loja, on=["cod_pro", "cd_loja"], how="inner")
print("Linhas do df após filtro alto giro:", df.shape)


Pares produto+loja alto giro (demanda líquida 2025 > 108): 6287
Linhas do df após filtro alto giro: (2035456, 6)


In [13]:
dem_diaria = (
    df.groupby(["cod_pro", "cd_loja", "dt_emissao"], as_index=False, observed=True)
      .agg(demanda_dia=("demanda_dia", "sum"))
      .sort_values(["cod_pro", "cd_loja", "dt_emissao"])
)

dem_diaria = dem_diaria.set_index("dt_emissao")

dem_diaria["demanda_LT"] = (
    dem_diaria.groupby(["cod_pro", "cd_loja"], observed=True)["demanda_dia"]
              .rolling(f"{LEAD_TIME}D")
              .sum()
              .reset_index(level=[0, 1], drop=True)
)

dem_lt = dem_diaria.reset_index()

In [14]:
min_dt = dem_lt.groupby(["cod_pro", "cd_loja"], observed=True)["dt_emissao"].transform("min")
dem_lt = dem_lt[dem_lt["dt_emissao"] >= (min_dt + pd.Timedelta(days=LEAD_TIME - 1))].copy()

produtos_alvo = dem_lt[["cod_pro", "cd_loja"]].drop_duplicates()

In [15]:
df_filtrado = df.merge(produtos_alvo, on=["cod_pro", "cd_loja"], how="inner").copy()
df_filtrado["dt_emissao"] = pd.to_datetime(df_filtrado["dt_emissao"])

vendas_mov = df_filtrado.loc[df_filtrado["vendas"] > 0, ["dt_emissao", "cod_pro", "cd_loja", "vendas"]].copy()
vendas_mov = vendas_mov.rename(columns={"vendas": "qtde"})
vendas_mov["qtde"] = -vendas_mov["qtde"]  # venda como saída

dev_mov = df_filtrado.loc[df_filtrado["devolucoes"] > 0, ["dt_emissao", "cod_pro", "cd_loja", "devolucoes"]].copy()
dev_mov = dev_mov.rename(columns={"devolucoes": "qtde"})  # devolução como entrada

df_movimentos = (
    pd.concat(
        [vendas_mov[["dt_emissao", "cod_pro", "cd_loja", "qtde"]],
         dev_mov[["dt_emissao", "cod_pro", "cd_loja", "qtde"]]],
        ignore_index=True
    )
    .sort_values(["cod_pro", "cd_loja", "dt_emissao"])
    .reset_index(drop=True)
)



In [16]:
from collections import defaultdict, deque
def simular_loss_fast(mov_df: pd.DataFrame, S: int, lead_time_dias: int = 30, inicio: str = "2025-01-01"):
    g = mov_df.copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"])
    g = g[g["dt_emissao"] >= pd.to_datetime(inicio)].sort_values("dt_emissao")
    if g.empty:
        return 0.0, 0.0

    dias = g["dt_emissao"].dt.normalize().astype("int64") // 86_400_000_000_000
    q = g["qtde"].to_numpy(dtype=np.float32)

    estoque = int(S)
    perdida = 0.0
    pedidos = defaultdict(int)  # dia_int -> qtd

    for d, qt in zip(dias, q):
        # aplica TODAS as chegadas vencidas até o dia d
        if pedidos:
            vencidos = [k for k in pedidos.keys() if k <= d]
            if vencidos:
                for k in sorted(vencidos):
                    if estoque >= S:
                        break
                    ent = min(pedidos[k], S - estoque)
                    if ent > 0:
                        estoque += int(ent)
                        pedidos[k] -= int(ent)
                    if pedidos.get(k, 0) <= 0:
                        pedidos.pop(k, None)

        if qt > 0:  # devolução
            ent = min(qt, S - estoque)
            estoque += int(ent)
        else:       # venda
            demanda = -qt
            if estoque >= demanda:
                estoque -= int(demanda)
                pedidos[d + lead_time_dias] += int(demanda)
            else:
                perdida += float(demanda - estoque)
                vendido = int(estoque)
                estoque = 0
                pedidos[d + lead_time_dias] += int(vendido)

    demanda_total = float((-q[q < 0]).sum())
    return float(perdida), float(demanda_total)

In [17]:
def loss_rate_para_S_fast(grupo_mov: pd.DataFrame, S: int, lead_time: int = 30, inicio: str = "2025-01-01") -> float:
    perdida, demanda_total = simular_loss_fast(grupo_mov[["dt_emissao", "qtde"]], int(S), lead_time, inicio)
    return 0.0 if demanda_total <= 0 else float(perdida / demanda_total)

In [18]:
def achar_S_para_loss_rate(
    grupo_mov: pd.DataFrame,
    alvo_loss: float = 0.05,
    lead_time: int = 30,
    inicio: str = "2025-01-01",
    S_min: int = 0,
    S_max: int | None = None,
) -> tuple[int, float]:

    g = grupo_mov.copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"])
    g = g[g["dt_emissao"] >= pd.to_datetime(inicio)].sort_values("dt_emissao")
    if g.empty:
        return 0, 0.0

    demanda_total = float((-g.loc[g["qtde"] < 0, "qtde"]).sum())
    if demanda_total <= 0:
        return 0, 0.0

    if S_max is None:
        dias = (g["dt_emissao"].max() - g["dt_emissao"].min()).days + 1
        media_dia = demanda_total / max(dias, 1)
        S_max = int(max(10, np.ceil(media_dia * lead_time * 2.0)))  # bound mais esperto que *5

    lo, hi = int(S_min), int(S_max)

    # garante hi suficiente (poucas tentativas)
    lr_hi = loss_rate_para_S_fast(g, hi, lead_time, inicio)
    tentativas = 0
    while lr_hi > alvo_loss and tentativas < 6:
        hi *= 2
        lr_hi = loss_rate_para_S_fast(g, hi, lead_time, inicio)
        tentativas += 1

    if lr_hi > alvo_loss:
        return hi, float(lr_hi)

    # busca binária com cache simples
    cache = {}
    def lr(S):
        S = int(S)
        if S not in cache:
            cache[S] = float(loss_rate_para_S_fast(g, S, lead_time, inicio))
        return cache[S]

    while lo < hi:
        mid = (lo + hi) // 2
        if lr(mid) <= alvo_loss:
            hi = mid
        else:
            lo = mid + 1

    return lo, lr(lo)


In [19]:
#quando ja tiver os estoques e nao quer esperar muito tempo (demora cerca de 5min):
#df_S_otimo = pd.read_parquet("df_S_otimo.parquet")

In [20]:
from collections import deque

def simular_estoque_ledger(
    movimentos_df: pd.DataFrame,
    estoque_max: int,
    lead_time_dias: int = 30,
    data_inicio_simulacao: str = "2025-01-01",
    ordenar_eventos: tuple[str, ...] = ("reabastecimento", "devolucao", "venda"),
) -> pd.DataFrame:

    dfm = movimentos_df.copy()
    dfm["dt_emissao"] = pd.to_datetime(dfm["dt_emissao"])
    inicio = pd.to_datetime(data_inicio_simulacao)
    dfm = dfm[dfm["dt_emissao"] >= inicio].sort_values("dt_emissao").reset_index(drop=True)

    if dfm.empty:
        return pd.DataFrame(columns=[
            "dt_emissao", "movimento", "qtde",
            "estoque_inicial", "estoque_final",
            "venda_perdida", "origem_sale_dt",
            "cod_pro", "cd_loja"
        ])

    dfm["dia"] = dfm["dt_emissao"].dt.normalize()

    cod_pro = dfm["cod_pro"].iloc[0] if "cod_pro" in dfm.columns else None
    cd_loja = dfm["cd_loja"].iloc[0] if "cd_loja" in dfm.columns else None

    pedidos = deque()  # FIFO de pedidos (cada um com origem)
    estoque = int(estoque_max)  # inicial cheio
    historico = []
    sale_seq = 0

    dias_mov = dfm["dia"].drop_duplicates()

    # como as reposições são geradas durante o loop, a gente precisa simular por um calendário contínuo
    # entre o primeiro e o último dia de movimento + lead_time
    dia_min = dias_mov.min()
    dia_max = dias_mov.max() + pd.Timedelta(days=lead_time_dias)
    dias_unicos = pd.date_range(dia_min, dia_max, freq="D")
    ordem = {nome: i for i, nome in enumerate(ordenar_eventos)}

    mov_por_dia = {d: g for d, g in dfm.groupby("dia", sort=False)}

    for dia in dias_unicos:
        # 1) aplica reposições vencidas até o dia
        while pedidos and pedidos[0]["arrival_dt"] <= dia:
            ped = pedidos[0]
            if estoque >= estoque_max:
                break
            cabe = estoque_max - estoque
            entra = min(int(ped["qty_remaining"]), int(cabe))
            if entra <= 0:
                break

            estoque_inicial = estoque
            estoque += int(entra)
            ped["qty_remaining"] -= int(entra)

            historico.append({
                "dt_emissao": dia,
                "movimento": "reabastecimento",
                "qtde": float(entra),
                "estoque_inicial": int(estoque_inicial),
                "estoque_final": int(estoque),
                "venda_perdida": 0.0,
                "origem_sale_dt": ped["sale_dt"],
                "cod_pro": cod_pro,
                "cd_loja": cd_loja,
            })

            if ped["qty_remaining"] <= 0:
                pedidos.popleft()

        # 2) processa movimentos do dia
        mov_por_dia = {d: g for d, g in dfm.groupby("dia", sort=False)}

        # dentro do loop
        mov_dia = mov_por_dia.get(dia)
        if mov_dia is not None:
            for _, row in mov_dia.iterrows():
                qtde = float(row["qtde"])

            if qtde > 0:  # devolução
                estoque_inicial = estoque
                entra = min(int(qtde), int(estoque_max - estoque))
                estoque += int(entra)

                historico.append({
                    "dt_emissao": dia,
                    "movimento": "devolucao",
                    "qtde": float(qtde),
                    "estoque_inicial": int(estoque_inicial),
                    "estoque_final": int(estoque),
                    "venda_perdida": 0.0,
                    "origem_sale_dt": None,
                    "cod_pro": cod_pro,
                    "cd_loja": cd_loja,
                })
                continue

            # venda
            demanda = int(round(-qtde))
            estoque_inicial = estoque

            if estoque >= demanda:
                vendido = demanda
                perdido = 0
                estoque -= int(vendido)
            else:
                vendido = int(estoque)
                perdido = int(demanda - estoque)
                estoque = 0

            historico.append({
                "dt_emissao": dia,
                "movimento": "venda",
                "qtde": float(qtde),
                "estoque_inicial": int(estoque_inicial),
                "estoque_final": int(estoque),
                "venda_perdida": float(perdido),
                "origem_sale_dt": None,
                "cod_pro": cod_pro,
                "cd_loja": cd_loja,
            })

            if vendido > 0:
                sale_seq += 1
                sale_id = f"{dia.strftime('%Y%m%d')}-{sale_seq:04d}"
                arrival = dia + pd.Timedelta(days=lead_time_dias)

                pedidos.append({
                    "arrival_dt": arrival,
                    "qty_total": int(vendido),
                    "qty_remaining": int(vendido),
                    "sale_dt": dia,
                })

        # 3) se devolução abriu espaço, tenta entrar mais reposição vencida hoje
        while pedidos and pedidos[0]["arrival_dt"] <= dia and estoque < estoque_max:
            ped = pedidos[0]
            cabe = estoque_max - estoque
            entra = min(int(ped["qty_remaining"]), int(cabe))
            if entra <= 0:
                break

            estoque_inicial = estoque
            estoque += int(entra)
            ped["qty_remaining"] -= int(entra)

            historico.append({
                "dt_emissao": dia,
                "movimento": "reabastecimento",
                "qtde": float(entra),
                "estoque_inicial": int(estoque_inicial),
                "estoque_final": int(estoque),
                "venda_perdida": 0.0,
                "origem_sale_dt": ped["sale_dt"],
                "cod_pro": cod_pro,
                "cd_loja": cd_loja,
            })

            if ped["qty_remaining"] <= 0:
                pedidos.popleft()

    out = pd.DataFrame(historico)
    out["mov_ord"] = out["movimento"].map(ordem).fillna(999).astype(int)
    out = out.sort_values(["dt_emissao", "mov_ord"]).drop(columns=["mov_ord"]).reset_index(drop=True)
    return out

In [21]:
HORIZON_DIAS = 70

def loss_rate_para_S_70d_fast(
    grupo_mov: pd.DataFrame,
    S: int,
    lead_time: int = 30,
    inicio: str = "2025-01-01",
    horizonte_dias: int = 70,
) -> float:
    g = grupo_mov.copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"])

    dt_ini = pd.to_datetime(inicio)
    dt_fim = dt_ini + pd.Timedelta(days=horizonte_dias)

    g = g[(g["dt_emissao"] >= dt_ini) & (g["dt_emissao"] < dt_fim)].sort_values("dt_emissao")
    if g.empty:
        return 0.0

    perdida, demanda_total = simular_loss_fast(g[["dt_emissao", "qtde"]], int(S), lead_time, inicio)
    return 0.0 if demanda_total <= 0 else float(perdida / demanda_total)


def achar_S_para_loss_70d_fast(
    grupo_mov: pd.DataFrame,
    alvo_loss: float = 0.05,
    lead_time: int = 30,
    inicio: str = "2025-01-01",
    horizonte_dias: int = 70,
    S_min: int = 0,
    S_max: int | None = None,
) -> tuple[int, float]:

    g = grupo_mov.copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"])

    dt_ini = pd.to_datetime(inicio)
    dt_fim = dt_ini + pd.Timedelta(days=horizonte_dias)
    g = g[(g["dt_emissao"] >= dt_ini) & (g["dt_emissao"] < dt_fim)].sort_values("dt_emissao")

    if g.empty:
        return 0, 0.0

    demanda_total = float((-g.loc[g["qtde"] < 0, "qtde"]).sum())
    if demanda_total <= 0:
        return 0, 0.0

    if S_max is None:
        dias = max((g["dt_emissao"].max() - g["dt_emissao"].min()).days + 1, 1)
        media_dia = demanda_total / dias
        S_max = int(max(10, np.ceil(media_dia * horizonte_dias * 1.5)))

    lo, hi = int(S_min), int(S_max)

    cache = {}
    def lr(S: int) -> float:
        S = int(S)
        if S not in cache:
            cache[S] = loss_rate_para_S_70d_fast(g, S, lead_time, inicio, horizonte_dias)
        return float(cache[S])

    lr_hi = lr(hi)
    tent = 0
    while lr_hi > alvo_loss and tent < 8:
        hi *= 2
        lr_hi = lr(hi)
        tent += 1

    if lr_hi > alvo_loss:
        return hi, float(lr_hi)

    while lo < hi:
        mid = (lo + hi) // 2
        if lr(mid) <= alvo_loss:
            hi = mid
        else:
            lo = mid + 1

    return lo, float(lr(lo))

In [22]:
import os
resultados = []

grupos = df_movimentos.groupby(["cod_pro", "cd_loja"], sort=False)
total = grupos.ngroups

for i, ((prod, loja), grupo) in enumerate(grupos, start=1):
    S, lr = achar_S_para_loss_70d_fast(
    grupo_mov=grupo,
    alvo_loss=ALVO_LOSS,
    lead_time=LEAD_TIME,
    inicio=DATA_INICIO_SIM,
    horizonte_dias=70,
)

    resultados.append({
        "cod_pro": str(prod),
        "cd_loja": str(loja),
        "S_ate_5pct_venda_perdida_70d_pior_janela": int(S),
        "pior_loss_70d": float(lr),
        "fill_rate_70d_pior_janela": float(1 - lr),
    })

    if i % 200 == 0:
        print(f"{i}/{total} grupos processados...")

df_S_otimo = (
    pd.DataFrame(resultados)
      .sort_values(
          ["pior_loss_70d", "S_ate_5pct_venda_perdida_70d_pior_janela"],
          ascending=[True, True]
      )
      .reset_index(drop=True)
)

# salva
out_path = os.path.join(os.getcwd(), "df_S_otimo.parquet")
df_S_otimo.to_parquet(out_path, index=False)
print("Salvo em:", out_path)

200/6281 grupos processados...
400/6281 grupos processados...
600/6281 grupos processados...
800/6281 grupos processados...
1000/6281 grupos processados...
1200/6281 grupos processados...
1400/6281 grupos processados...
1600/6281 grupos processados...
1800/6281 grupos processados...
2000/6281 grupos processados...
2200/6281 grupos processados...
2400/6281 grupos processados...
2600/6281 grupos processados...
2800/6281 grupos processados...
3000/6281 grupos processados...
3200/6281 grupos processados...
3400/6281 grupos processados...
3600/6281 grupos processados...
3800/6281 grupos processados...
4000/6281 grupos processados...
4200/6281 grupos processados...
4400/6281 grupos processados...
4600/6281 grupos processados...
4800/6281 grupos processados...
5000/6281 grupos processados...
5200/6281 grupos processados...
5400/6281 grupos processados...
5600/6281 grupos processados...
5800/6281 grupos processados...
6000/6281 grupos processados...
6200/6281 grupos processados...
Salvo em: c:

In [23]:
df_movimentos["cod_pro"] = norm_codpro(df_movimentos["cod_pro"])
df_movimentos["cd_loja"] = norm_loja(df_movimentos["cd_loja"])

# --- 1) grid por SKU/loja (max) ---
grid_sim = df_grid.loc[:, ["codpro", "qt_grid", "cd_loja"]].copy()
grid_sim["cod_pro"] = norm_codpro(grid_sim["codpro"])
grid_sim["cd_loja"] = norm_loja(grid_sim["cd_loja"])
grid_sim["qt_grid"] = pd.to_numeric(grid_sim["qt_grid"], errors="coerce").fillna(0).astype("int32")
grid_sim = (
    grid_sim.loc[:, ["cod_pro", "cd_loja", "qt_grid"]]
            .groupby(["cod_pro","cd_loja"], as_index=False, observed=True)
            .agg(qt_grid=("qt_grid","max"))
)

# --- 2) base de comparação: S_otimo + grid ---
base_S = df_S_otimo.copy()
base_S["cod_pro"] = norm_codpro(base_S["cod_pro"])
base_S["cd_loja"] = norm_loja(base_S["cd_loja"])

base_S = base_S.merge(grid_sim, on=["cod_pro","cd_loja"], how="left")
base_S["qt_grid"] = base_S["qt_grid"].fillna(0).astype("int32")

# --- 3) calcula loss com S = grid e com S = S_otimo ---
result = []

mov_groups = df_movimentos.groupby(["cod_pro","cd_loja"], sort=False)

for (prod, loja), g in mov_groups:
    g = g[g["dt_emissao"] >= pd.to_datetime(DATA_INICIO_SIM)].copy()
    if g.empty:
        continue

    rowS = base_S[(base_S["cod_pro"] == prod) & (base_S["cd_loja"] == loja)]
    if rowS.empty:
        continue

    S_otimo = int(rowS["S_ate_5pct_venda_perdida_70d_pior_janela"].iloc[0])
    S_grid  = int(rowS["qt_grid"].iloc[0])

    
    loss_grid  = float(loss_rate_para_S_70d_fast(g, S_grid,  LEAD_TIME, DATA_INICIO_SIM, 70))
    loss_otimo = float(loss_rate_para_S_70d_fast(g, S_otimo, LEAD_TIME, DATA_INICIO_SIM, 70))

    demanda_total = float((-g.loc[g["qtde"] < 0, "qtde"]).sum())

    result.append({
        "cod_pro": str(prod),
        "cd_loja": str(loja),
        "S_grid": S_grid,
        "S_otimo": S_otimo,
        "loss_grid": loss_grid,
        "loss_otimo": loss_otimo,
        "delta_loss": loss_grid - loss_otimo,
        "demanda_total": demanda_total
    })

df_comp_servico = pd.DataFrame(result)


In [24]:
print("Top 20 onde o S ótimo mais reduz perda (delta_loss alto):")
print(df_comp_servico.sort_values("delta_loss", ascending=False).head(10))

Top 20 onde o S ótimo mais reduz perda (delta_loss alto):
     cod_pro cd_loja  S_grid  S_otimo  loss_grid  loss_otimo  delta_loss  \
2335  026856      05       0        8        1.0         0.0         1.0   
6072  089856      05       0       16        1.0         0.0         1.0   
1915  023901      07       0       15        1.0         0.0         1.0   
2173  025530      04       0       10        1.0         0.0         1.0   
431   013462      05       0        8        1.0         0.0         1.0   
4858  061520      04       0       12        1.0         0.0         1.0   
5565  076719      07       0       17        1.0         0.0         1.0   
4826  060743      07       0        9        1.0         0.0         1.0   
5059  065615      06       0        3        1.0         0.0         1.0   
1998  024466      04       0       10        1.0         0.0         1.0   

      demanda_total  
2335          123.0  
6072          120.0  
1915          118.0  
2173          117

In [25]:
print("\nTop 20 onde a grid é melhor/igual (delta_loss <= 0):")
print(df_comp_servico.sort_values("delta_loss", ascending=True).head(10))


Top 20 onde a grid é melhor/igual (delta_loss <= 0):
     cod_pro cd_loja  S_grid  S_otimo  loss_grid  loss_otimo  delta_loss  \
2270  026433      06      42       12        0.0        0.10       -0.10   
5716  080404      03      40       37        0.0        0.05       -0.05   
5755  081415      04      35       32        0.0        0.05       -0.05   
5721  080408      03      20       19        0.0        0.05       -0.05   
1476  021721      04      16       15        0.0        0.05       -0.05   
4429  058307      03      20       19        0.0        0.05       -0.05   
6133  092132      04      24       14        0.0        0.05       -0.05   
3502  040474      04     100       90        0.0        0.05       -0.05   
4433  058307      07      90       84        0.0        0.05       -0.05   
4494  058342      05      20       19        0.0        0.05       -0.05   

      demanda_total  
2270          127.0  
5716          460.0  
5755          351.0  
5721          210.0  

In [26]:
def refinar_S_minimo_ate_5(grupo_mov: pd.DataFrame,
                            S_inicial: int,
                            alvo_loss: float = 0.05,
                            lead_time: int = 30,
                            inicio: str = "2025-01-01",
                            horizonte_dias: int = 70
                        ) -> dict:
    """
    Desce o S a partir de S_inicial até encontrar o menor S que ainda mantém loss <= alvo_loss.
    Retorna também o S "quebra" (um abaixo) e o loss dele, pra você enxergar o ponto de virada.
    """
    g = grupo_mov.copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"])
    g = g[g["dt_emissao"] >= pd.to_datetime(inicio)].sort_values("dt_emissao")
    if g.empty:
        return {"S_ok": 0, "loss_ok": 0.0, "S_quebra": None, "loss_quebra": None}

    cache = {}
    def lr(S: int) -> float:
        S = int(max(0, S))
        if S not in cache:
            cache[S] = float(loss_rate_para_S_70d_fast(g, S, lead_time, inicio, horizonte_dias))
        return cache[S]

    S = int(max(0, S_inicial))
    loss_S = lr(S)

    # se por algum motivo S_inicial não estiver ok, sobe até ficar ok (robustez)
    tent = 0
    while loss_S > alvo_loss and tent < 50:
        S += 1
        loss_S = lr(S)
        tent += 1

    # agora desce até o limite
    while S > 0:
        loss_prev = lr(S - 1)
        if loss_prev <= alvo_loss:
            S -= 1
            loss_S = loss_prev
        else:
            return {
                "S_ok": int(S),
                "loss_ok": float(loss_S),
                "S_quebra": int(S - 1),
                "loss_quebra": float(loss_prev),
            }

    return {"S_ok": int(S), "loss_ok": float(loss_S), "S_quebra": None, "loss_quebra": None}

In [27]:
resultados_refino_5 = []

# normaliza chaves (importante)
df_S_otimo["cod_pro"] = norm_codpro(df_S_otimo["cod_pro"])
df_S_otimo["cd_loja"] = norm_loja(df_S_otimo["cd_loja"])
df_movimentos["cod_pro"] = norm_codpro(df_movimentos["cod_pro"])
df_movimentos["cd_loja"] = norm_loja(df_movimentos["cd_loja"])

grupos = df_movimentos.groupby(["cod_pro", "cd_loja"], sort=False)
total = grupos.ngroups

for i, ((prod, loja), grupo) in enumerate(grupos, start=1):
    row = df_S_otimo[(df_S_otimo["cod_pro"] == prod) & (df_S_otimo["cd_loja"] == loja)]
    if row.empty:
        continue

    S_5pct = int(row["S_ate_5pct_venda_perdida_70d_pior_janela"].iloc[0])

    r = refinar_S_minimo_ate_5(
        grupo_mov=grupo,
        S_inicial=S_5pct,
        alvo_loss=0.05,
        lead_time=LEAD_TIME,
        inicio=DATA_INICIO_SIM,
        horizonte_dias=70
    )

    resultados_refino_5.append({
        "cod_pro": str(prod),
        "cd_loja": str(loja),
        "S_5pct_original": int(S_5pct),
        "S_5pct_refinado": int(r["S_ok"]),
        "loss_refinado": float(r["loss_ok"]),
        "S_quebra": r["S_quebra"],
        "loss_quebra": r["loss_quebra"],
        "economia_unidades": int(S_5pct - r["S_ok"]),
    })

    if i % 200 == 0:
        print(f"{i}/{total} refinados (5%)...")

df_S_5_refinado = pd.DataFrame(resultados_refino_5)

200/6281 refinados (5%)...
400/6281 refinados (5%)...
600/6281 refinados (5%)...
800/6281 refinados (5%)...
1000/6281 refinados (5%)...
1200/6281 refinados (5%)...
1400/6281 refinados (5%)...
1600/6281 refinados (5%)...
1800/6281 refinados (5%)...
2000/6281 refinados (5%)...
2200/6281 refinados (5%)...
2400/6281 refinados (5%)...
2600/6281 refinados (5%)...
2800/6281 refinados (5%)...
3000/6281 refinados (5%)...
3200/6281 refinados (5%)...
3400/6281 refinados (5%)...
3600/6281 refinados (5%)...
3800/6281 refinados (5%)...
4000/6281 refinados (5%)...
4200/6281 refinados (5%)...
4400/6281 refinados (5%)...
4600/6281 refinados (5%)...
4800/6281 refinados (5%)...
5000/6281 refinados (5%)...
5200/6281 refinados (5%)...
5400/6281 refinados (5%)...
5600/6281 refinados (5%)...
5800/6281 refinados (5%)...
6000/6281 refinados (5%)...
6200/6281 refinados (5%)...


In [28]:
df_S_5_refinado

,cod_pro,cd_loja,S_5pct_original,S_5pct_refinado,loss_refinado,S_quebra,loss_quebra,economia_unidades
0,000136,05,15,15,0.038462,14.0,0.076923,0
1,000276,04,66,66,0.050000,65.0,0.062500,0
2,000276,05,57,57,0.050000,56.0,0.066667,0
3,000276,06,19,19,0.050000,18.0,0.100000,0
4,000276,07,10,10,0.000000,9.0,0.100000,0
...,...,...,...,...,...,...,...,...
6276,109763,04,0,0,0.000000,NaN,NaN,0
6277,109763,05,0,0,0.000000,NaN,NaN,0
6278,109763,06,0,0,0.000000,NaN,NaN,0
6279,109763,07,0,0,0.000000,NaN,NaN,0


In [29]:
df_economia = df_S_5_refinado[df_S_5_refinado["economia_unidades"] > 0].copy()

print("Qtd de pares com economia > 0:", len(df_economia))
df_economia.sort_values("economia_unidades", ascending=False).head(50)

Qtd de pares com economia > 0: 19


,cod_pro,cd_loja,S_5pct_original,S_5pct_refinado,loss_refinado,S_quebra,loss_quebra,economia_unidades
5371,070578,04,412,408,0.048780,407.0,0.050813,4
4010,055784,07,23,20,0.047619,19.0,0.071429,3
3744,047664,06,30,27,0.036364,26.0,0.054545,3
49,002501,04,38,36,0.049180,35.0,0.065574,2
1556,021910,03,23,21,0.031250,20.0,0.062500,2
4805,060334,03,63,61,0.041096,60.0,0.054795,2
4562,059247,06,20,18,0.031250,17.0,0.062500,2
2628,028365,03,13,12,0.033333,11.0,0.066667,1
3864,051627,05,10,9,0.050000,8.0,0.100000,1
3131,031537,03,52,51,0.046729,50.0,0.056075,1


In [41]:
def auditar_S(prod: str, loja: str, modo: str = "S"):
    prod = str(prod)
    loja = str(loja)
    modo = str(modo).strip().lower()

    # normaliza chaves
    prod = norm_codpro(pd.Series([prod])).iloc[0]
    loja = norm_loja(pd.Series([loja])).iloc[0]

    # normaliza df_S_otimo (garante match)
    dfS = df_S_otimo.copy()
    dfS["cod_pro"] = norm_codpro(dfS["cod_pro"])
    dfS["cd_loja"] = norm_loja(dfS["cd_loja"])

    srow = dfS.loc[(dfS["cod_pro"] == prod) & (dfS["cd_loja"] == loja)]
    if srow.empty:
        print("Não achei esse produto/loja em df_S_otimo")
        return pd.DataFrame()

    S_otimo = int(srow["S_ate_5pct_venda_perdida_70d_pior_janela"].iloc[0])

    # pega S_refinado (df_S_5_refinado) - só muda o df, lógica igual
    S_refinado = None
    if "df_S_5_refinado" in globals():
        dfR = df_S_5_refinado.copy()
        dfR["cod_pro"] = norm_codpro(dfR["cod_pro"])
        dfR["cd_loja"] = norm_loja(dfR["cd_loja"])
        rrow = dfR.loc[(dfR["cod_pro"] == prod) & (dfR["cd_loja"] == loja)]
        if not rrow.empty:
            S_refinado = int(rrow["S_5pct_refinado"].iloc[0])

    # pega S_grid da df_grid
    grid_sim = df_grid.loc[:, ["codpro", "qt_grid", "cd_loja"]].copy()
    grid_sim["cod_pro"] = norm_codpro(grid_sim["codpro"])
    grid_sim["cd_loja"] = norm_loja(grid_sim["cd_loja"])
    grid_sim["qt_grid"] = pd.to_numeric(grid_sim["qt_grid"], errors="coerce").fillna(0).astype("int32")
    grid_sim = (
        grid_sim.loc[:, ["cod_pro", "cd_loja", "qt_grid"]]
                .groupby(["cod_pro", "cd_loja"], as_index=False, observed=True)
                .agg(qt_grid=("qt_grid", "max"))
    )

    grow = grid_sim.loc[(grid_sim["cod_pro"] == prod) & (grid_sim["cd_loja"] == loja)]
    S_grid = int(grow["qt_grid"].iloc[0]) if not grow.empty else 0

    # escolhe qual S usar
    if modo in ("grid", "g"):
        S_usado = S_grid
        label = "GRID"
    elif modo in ("ref", "r", "refinado"):
        if S_refinado is None:
            print("Não achei esse produto/loja em df_S_5_refinado (ou df_S_5_refinado não existe).")
            return pd.DataFrame()
        S_usado = S_refinado
        label = "S_REFINADO_5%"
    else:
        raise ValueError("modo inválido. Use 'grid' ou 'ref'.")

    # movimentos do par
    g = df_movimentos[(df_movimentos["cod_pro"] == prod) & (df_movimentos["cd_loja"] == loja)].copy()
    g["dt_emissao"] = pd.to_datetime(g["dt_emissao"])

    dt_ini = pd.to_datetime(DATA_INICIO_SIM)
    dt_fim = dt_ini + pd.Timedelta(days=HORIZON_DIAS)

    g = g[(g["dt_emissao"] >= dt_ini) & (g["dt_emissao"] < dt_fim)].sort_values("dt_emissao")
    if g.empty:
        print(f"Sem movimentos no horizonte de {HORIZON_DIAS} dias")
        return pd.DataFrame()

    # roda ledger com o S escolhido
    sim = simular_estoque_ledger(
        movimentos_df=g[["dt_emissao", "cod_pro", "cd_loja", "qtde"]],
        estoque_max=int(S_usado),
        lead_time_dias=LEAD_TIME,
        data_inicio_simulacao=DATA_INICIO_SIM
    )

    # métricas (70 dias)
    demanda_total = float((-g.loc[g["qtde"] < 0, "qtde"]).sum())
    perdida = float(sim["venda_perdida"].sum())
    loss = (perdida / demanda_total) if demanda_total > 0 else 0.0
    fill = 1 - loss

    loss_fast = float(loss_rate_para_S_70d_fast(g, S_usado, LEAD_TIME, DATA_INICIO_SIM, HORIZON_DIAS))

    media_dia = demanda_total / HORIZON_DIAS if HORIZON_DIAS > 0 else 0.0
    dias_cobertura = (S_usado / media_dia) if media_dia > 0 else np.nan

    print(f"Produto {prod} | Loja {loja} | Modo: {label}")
    print(f"S usado: {S_usado}  (grid={S_grid} | refinado={S_refinado if S_refinado is not None else 'NA'})")
    print(f"Horizonte: {HORIZON_DIAS} dias a partir de {DATA_INICIO_SIM}")
    print(f"Demanda total (70d): {demanda_total:.0f}")
    print(f"Venda perdida (70d): {perdida:.0f}")
    print(f"Loss rate (ledger 70d): {loss:.2%}")
    print(f"Média/dia (70d): {media_dia:.2f} | Cobertura do S: {dias_cobertura:.1f} dias")

    cols_final = [
    "cod_pro","cd_loja","dt_emissao","estoque_inicial","qtde",
    "estoque_final","venda_perdida","movimento","origem_sale_dt"
]
    return sim.loc[:, cols_final]


In [42]:
auditar_S("062470", "04", "grid") 
auditar_S("062470", "04", "ref")  

Produto 062470 | Loja 04 | Modo: GRID
S usado: 16  (grid=16 | refinado=15)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 24
Venda perdida (70d): 0
Loss rate (ledger 70d): 0.00%
Média/dia (70d): 0.34 | Cobertura do S: 46.7 dias
Produto 062470 | Loja 04 | Modo: S_REFINADO_5%
S usado: 15  (grid=16 | refinado=15)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 24
Venda perdida (70d): 1
Loss rate (ledger 70d): 4.17%
Média/dia (70d): 0.34 | Cobertura do S: 43.8 dias


,cod_pro,cd_loja,dt_emissao,estoque_inicial,qtde,estoque_final,venda_perdida,movimento,origem_sale_dt
0,062470,04,2025-01-06,15,-8.0,7,0.0,venda,NaT
1,062470,04,2025-01-08,7,-8.0,0,1.0,venda,NaT
2,062470,04,2025-01-09,0,8.0,8,0.0,devolucao,NaT
3,062470,04,2025-02-05,8,7.0,15,0.0,reabastecimento,2025-01-06
4,062470,04,2025-02-10,7,1.0,8,0.0,reabastecimento,2025-01-06
5,062470,04,2025-02-10,8,7.0,15,0.0,reabastecimento,2025-01-08
6,062470,04,2025-02-10,15,-8.0,7,0.0,venda,NaT


In [ ]:
auditar_S("019561", "06", "grid") 

Produto 019561 | Loja 06 | Modo: GRID
S usado: 6  (grid=6 | refinado=18)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 22
Venda perdida (70d): 13
Loss rate (ledger 70d): 59.09%
Média/dia (70d): 0.31 | Cobertura do S: 19.1 dias
Produto 019561 | Loja 06 | Modo: S_REFINADO_5%
S usado: 18  (grid=6 | refinado=18)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 22
Venda perdida (70d): 1
Loss rate (ledger 70d): 4.55%
Média/dia (70d): 0.31 | Cobertura do S: 57.3 dias


,cod_pro,cd_loja,dt_emissao,estoque_inicial,qtde,estoque_final,venda_perdida,movimento,origem_sale_dt
0,019561,06,2025-01-03,18,-1.0,17,0.0,venda,NaT
1,019561,06,2025-01-06,17,-1.0,16,0.0,venda,NaT
2,019561,06,2025-01-13,16,-8.0,8,0.0,venda,NaT
3,019561,06,2025-01-21,8,-4.0,4,0.0,venda,NaT
4,019561,06,2025-02-02,4,1.0,5,0.0,reabastecimento,2025-01-03
5,019561,06,2025-02-03,5,-2.0,3,0.0,venda,NaT
6,019561,06,2025-02-04,3,-4.0,0,1.0,venda,NaT
7,019561,06,2025-02-05,0,1.0,1,0.0,reabastecimento,2025-01-06
8,019561,06,2025-02-12,1,8.0,9,0.0,reabastecimento,2025-01-13
9,019561,06,2025-02-13,9,-1.0,8,0.0,venda,NaT


In [33]:
auditar_S("019561", "06", "ref")  

Produto 019561 | Loja 06 | Modo: S_REFINADO_5%
S usado: 18  (grid=6 | ótimo=18 | refinado=18)
Horizonte: 70 dias a partir de 2025-01-01
Demanda total (70d): 22
Venda perdida (70d): 1
Loss rate (ledger 70d): 4.55%
Loss rate (fast 70d):   4.55%
Fill rate (70d): 95.45%
Média/dia (70d): 0.31 | Cobertura do S: 57.3 dias


,cod_pro,cd_loja,dt_emissao,estoque_inicial,qtde,estoque_final,venda_perdida,movimento,origem_sale_dt
0,019561,06,2025-01-03,18,-1.0,17,0.0,venda,NaT
1,019561,06,2025-01-06,17,-1.0,16,0.0,venda,NaT
2,019561,06,2025-01-13,16,-8.0,8,0.0,venda,NaT
3,019561,06,2025-01-21,8,-4.0,4,0.0,venda,NaT
4,019561,06,2025-02-02,4,1.0,5,0.0,reabastecimento,2025-01-03
5,019561,06,2025-02-03,5,-2.0,3,0.0,venda,NaT
6,019561,06,2025-02-04,3,-4.0,0,1.0,venda,NaT
7,019561,06,2025-02-05,0,1.0,1,0.0,reabastecimento,2025-01-06
8,019561,06,2025-02-12,1,8.0,9,0.0,reabastecimento,2025-01-13
9,019561,06,2025-02-13,9,-1.0,8,0.0,venda,NaT
